## 5.0 Data analysis

Data analysis will be conducted in this notebook and dataset used is the full dataset with all categorical features converted into model-understanding binary numerical columns. Several models will be implemented using GridSearch CV for the best performance. The discussion will emphasise on the most outstanding model selected to explain the fatality outcome based on the significant features found in the model. In closing, improvement and development for future analysis will be discussed in the last section of this analysis. 

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,auc,recall_score,precision_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix,roc_auc_score,cohen_kappa_score,make_scorer

Using TensorFlow backend.


In [3]:
data = pd.read_csv('datas/final_dataset.csv')
data.head()

,cas_age,n_occupants,total_units,cas_total,area_speed,cas_type_Driver,cas_type_Passenger,cas_type_Pedestrian,cas_type_Rider,cas_gender_Female,...,crash_type_Right Turn,crash_type_Roll Over,crash_type_Side Swipe,traf_ctrls_Give Way Sign,traf_ctrls_No Control,traf_ctrls_Other,traf_ctrls_Roundabout,traf_ctrls_Stop Sign,traf_ctrls_Traffic Signals,fatality
0,34.000000,1.0,2,1,60,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,41.000000,1.0,2,1,60,1,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
2,39.889159,1.0,2,1,60,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,19.000000,1.0,2,7,60,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,48.000000,6.0,2,7,60,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
X = data.drop(['fatality'], axis=1)
y = data['fatality']

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

## 5.1 Model tuning

We want to tune the parameters for each model along with different sampling techniques using StratifiedKFold to avoid overfitting data in extremely imbalanced dataset. The study of models includes logistic regression, MLP classifier and RusBoostClassifier and resampling techiques such as upsampling and downsampling will be performed during model training to optimise the prediction power due to underepresented class in the dataset. 

Due to highly imbalanced classes in the dataset, conventional score metrics such as accurracy, recall score and precision score might be ineffective in filtering the best parameters for model tuning. As such, score metric will be implemented in GridSearchCV is Cohen Kappa score, which is highly effective to measure how close an observed accuracy predicted in the selected model is compared with an expected accuracy. The closer the kappa score to 1, the better agreements between expected accuracy and observed accuracy in the model.

For usefulness of the model, test set will be split from the original dataset and it will only be used for testing purpose in section 5.2 to ease overfitting issue. 

In [6]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

# split test set for evaluation
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
near_miss = NearMiss(sampling_strategy='auto',random_state=51)
smote = SMOTE(sampling_strategy='auto',random_state=52)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=46,stratify=y,shuffle=True)

In [7]:
y_train.shape
y_test.shape

(3091,)

In [8]:
#PlotConfusionMatrix is to plot the confusion matrix and 
#normalised confusion matrix to visualise prediction for individual model
def PlotConfusionMatrix(y_test,pred):
    y_test_not_fatal=y_test.value_counts()[0]
    y_test_fatal=y_test.value_counts()[1]
    cfn_matrix = confusion_matrix(y_test,pred)
    cfn_norm_matrix = np.array([[1.0 / y_test_not_fatal,1.0/y_test_not_fatal],[1.0/y_test_fatal,1.0/y_test_fatal]])
    norm_cfn_matrix = cfn_matrix * cfn_norm_matrix

    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,2,1)
    sns.heatmap(cfn_matrix,cmap='coolwarm_r',linewidths=0.5,annot=True,ax=ax)
    plt.title('Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')

    ax = fig.add_subplot(1,2,2)
    sns.heatmap(norm_cfn_matrix,cmap='coolwarm_r',linewidths=0.5,annot=True,ax=ax)

    plt.title('Normalized Confusion Matrix')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
    plt.show()
    

## 5.1.1 Logistic regression

### Downsampling logistic regression

In [9]:
log_reg1 = LogisticRegression(random_state=2)
pipe_log_down = Pipeline([('sampling',near_miss), ('log',log_reg1)])
log_reg_params = {"log__penalty": ['l1', 'l2'], 'log__C': [0.0001,0.001, 0.01, 0.1, 1],
                 'log__class_weight':[{0:1,1:1},{0:1,1:10},{0:1,1:100},{0:1,1:1000}]}
log_reg_down_sampling = GridSearchCV(pipe_log_down, param_grid=log_reg_params, scoring=make_scorer(cohen_kappa_score),
                                   cv = skf)
log_reg_down_sampling.fit(X_train,y_train)
log_reg_down_sampling.best_params_

{'log__C': 0.0001, 'log__class_weight': {0: 1, 1: 10}, 'log__penalty': 'l2'}

### Upsampling logistic regression

In [10]:
log_reg2 = LogisticRegression(random_state=3)
pipe_log_up = Pipeline([('sampling',smote), ('log',log_reg2)])
log_reg_up_sampling = GridSearchCV(pipe_log_up, param_grid=log_reg_params, scoring=make_scorer(cohen_kappa_score),
                                   cv = skf)
log_reg_up_sampling.fit(X_train,y_train)
log_reg_up_sampling.best_params_

{'log__C': 0.1, 'log__class_weight': {0: 1, 1: 1}, 'log__penalty': 'l1'}

## 5.1.2  MLPClassifier

In [11]:
from sklearn.neural_network import MLPClassifier
mlp_params = {
    'nn__hidden_layer_sizes': [(50,50), (50,100), (100,)],
    'nn__activation': ['tanh', 'relu'],
    'nn__solver': ['sgd', 'adam'],
    'nn__alpha': [ 0.001,0.01],
    'nn__learning_rate': ['constant','adaptive']
}

### MLP downsampling

In [12]:
mlp1 = MLPClassifier(max_iter=200,random_state=4)
pipe_mlp_down = Pipeline([('sampling',near_miss), ('nn',mlp1)])
mlp_down_sampling = GridSearchCV(pipe_mlp_down, mlp_params, n_jobs=-1, cv=skf,scoring=make_scorer(cohen_kappa_score))
mlp_down_sampling.fit(X_train, y_train)
mlp_down_sampling.best_params_

{'nn__activation': 'tanh',
 'nn__alpha': 0.01,
 'nn__hidden_layer_sizes': (100,),
 'nn__learning_rate': 'constant',
 'nn__solver': 'sgd'}

### MLP upsampling

In [13]:
mlp2 = MLPClassifier(max_iter=200,random_state=5)
pipe_mlp_up = Pipeline([('sampling',smote), ('nn',mlp2)])
mlp_up_sampling = GridSearchCV(pipe_mlp_up, mlp_params, n_jobs=-1, cv=skf,scoring=make_scorer(cohen_kappa_score))
mlp_up_sampling.fit(X_train, y_train)
mlp_up_sampling.best_params_

{'nn__activation': 'relu',
 'nn__alpha': 0.001,
 'nn__hidden_layer_sizes': (50, 50),
 'nn__learning_rate': 'constant',
 'nn__solver': 'adam'}

## 5.1.3 RUSBoostClassifier

In [14]:
from imblearn.ensemble import RUSBoostClassifier
from sklearn.tree import DecisionTreeClassifier
rus = RUSBoostClassifier(sampling_strategy='auto',random_state=6)
pipe_rus = Pipeline([ ('rus',rus)])
rus_params = {'rus__learning_rate':[0.001,0.01,0.1,1],'rus__n_estimators':[100,200,300]}
rus_clf = GridSearchCV(pipe_rus, param_grid=rus_params, scoring=make_scorer(cohen_kappa_score),
                                   cv = skf)
rus_clf.fit(X_train,y_train)
rus_clf.best_params_

{'rus__learning_rate': 0.1, 'rus__n_estimators': 300}

## 5.2 Model selection
This section aims to select the best model out of all trained models in previous section. Test set split in section 5.1 will be used to measure how accurate the prediction is for each model. Score matrices will be performed include, Kappa score, recall score for class 1 to measure percentage of correctly predicted non-fatal casualties, recall score for class 0 to measure percentage of correctly predicted fatal casualties, confusion matrix, classification report and roc auc score to measure how good prediction overall. 

In [15]:
from sklearn.model_selection import cross_val_score
kappa=make_scorer(cohen_kappa_score)

### 5.2.1 Validation performance

In [16]:
def performance_eval1(model, test_X, test_y):
    y_pred = model.predict(test_X)
    print("Percentage of correct non-fatal outcome: ",round(recall_score(test_y,y_pred,pos_label=0),4)*100,"%")
    print("Percentage of correct fatal outcome: ",round(recall_score(test_y,y_pred),4)*100,"%")
    print("roc_auc_score: ",round(roc_auc_score(test_y,y_pred),4)*100,"%")
    print("------------Confusion matrix----------------------")
    print(confusion_matrix(test_y, y_pred,labels=[0,1]))
    print("--------------------------------------------------")
    print("---------------------------------Classification report-----------------------------")
    print(classification_report_imbalanced(test_y, y_pred))
    print("-----------------------------------------------------------------------------------")
    print("Cohen kappa score: ",round(cohen_kappa_score(test_y, y_pred,labels=[0,1])*100,2),"%")

In [17]:
performance_eval1(log_reg_down_sampling,X_test,y_test)

Percentage of correct non-fatal outcome:  48.03 %
Percentage of correct fatal outcome:  95.56 %
roc_auc_score:  71.78999999999999 %
------------Confusion matrix----------------------
[[1463 1583]
 [   2   43]]
--------------------------------------------------
---------------------------------Classification report-----------------------------
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.48      0.96      0.65      0.68      0.44      3046
          1       0.03      0.96      0.48      0.05      0.68      0.48        45

avg / total       0.98      0.49      0.95      0.64      0.68      0.44      3091

-----------------------------------------------------------------------------------
Cohen kappa score:  2.38 %


In [18]:
performance_eval1(log_reg_up_sampling,X_test,y_test)

Percentage of correct non-fatal outcome:  84.31 %
Percentage of correct fatal outcome:  62.22 %
roc_auc_score:  73.26 %
------------Confusion matrix----------------------
[[2568  478]
 [  17   28]]
--------------------------------------------------
---------------------------------Classification report-----------------------------
                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.84      0.62      0.91      0.72      0.54      3046
          1       0.06      0.62      0.84      0.10      0.72      0.51        45

avg / total       0.98      0.84      0.63      0.90      0.72      0.54      3091

-----------------------------------------------------------------------------------
Cohen kappa score:  7.7 %


In [19]:
performance_eval1(mlp_down_sampling,X_test,y_test)

Percentage of correct non-fatal outcome:  52.76 %
Percentage of correct fatal outcome:  86.67 %
roc_auc_score:  69.71000000000001 %
------------Confusion matrix----------------------
[[1607 1439]
 [   6   39]]
--------------------------------------------------
---------------------------------Classification report-----------------------------
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.53      0.87      0.69      0.68      0.44      3046
          1       0.03      0.87      0.53      0.05      0.68      0.47        45

avg / total       0.98      0.53      0.86      0.68      0.68      0.44      3091

-----------------------------------------------------------------------------------
Cohen kappa score:  2.36 %


In [20]:
performance_eval1(mlp_up_sampling,X_test,y_test)

Percentage of correct non-fatal outcome:  99.08 %
Percentage of correct fatal outcome:  11.110000000000001 %
roc_auc_score:  55.1 %
------------Confusion matrix----------------------
[[3018   28]
 [  40    5]]
--------------------------------------------------
---------------------------------Classification report-----------------------------
                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.99      0.11      0.99      0.33      0.12      3046
          1       0.15      0.11      0.99      0.13      0.33      0.10        45

avg / total       0.97      0.98      0.12      0.98      0.33      0.12      3091

-----------------------------------------------------------------------------------
Cohen kappa score:  11.73 %


In [21]:
performance_eval1(rus_clf,X_test,y_test)

Percentage of correct non-fatal outcome:  82.11 %
Percentage of correct fatal outcome:  77.78 %
roc_auc_score:  79.94 %
------------Confusion matrix----------------------
[[2501  545]
 [  10   35]]
--------------------------------------------------
---------------------------------Classification report-----------------------------
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.82      0.78      0.90      0.80      0.64      3046
          1       0.06      0.78      0.82      0.11      0.80      0.64        45

avg / total       0.98      0.82      0.78      0.89      0.80      0.64      3091

-----------------------------------------------------------------------------------
Cohen kappa score:  8.73 %


### 5.2.2 Discussion

**Test performance**

| Model         | Resampling | Percentage of correct non-fatal outcome |Percentage of correct fatal outcome|  roc auc  | kappa  |
|---------------|----------|-----------------------------------------|-----------------------------------|-------    |-----   |
|   Logistic    |   down   |                  48.03%                 |                95.56%             |   71.79%  |  2.38% |
|   Logistic    |    up    |                  84.31%                 |                62.22%             |   73.26%  |  7.70% |
|      MLP      |   down   |                  52.76%                 |                86.67%             |   69.71%  |  2.36% |
|      MLP      |    up    |                  99.08%                 |                11.11%             |   55.11%  | 11.73% |
|  RusBoosting  |   down   |                  82.11%                 |                77.78%             |   79.94%  |  8.73% |

Although mlp classifier with upsampling's kappa score is 11.73%, it poorly predicted (11.11%) 5 out of 45 fatal causalties for test set. This is due to the extremely imbalanced classes and high prediction on non-fatal casualtes. Logistic regression implemented with downsampling can correctly predict 43 out of 45 fatal casualties, but it has very high type 1 error, as percentage of correctly predicted non-fatal outcome is only 48.03 %.

The most desirable model is the RUSBoosting Classifier as percentage of correctly predicted fatal outcome is 77.78% and percentage of correctly predicted non-fatal outcome is 82.11%. Its roc aus score is 79.94%, suggesting the percentage of type 1 and type 2 errors are the lowest out of all models in previous section.

One notable result is that rus boosting classifier has very low kappa score. According to [score magnitude guideline](https://en.wikipedia.org/wiki/Cohen%27s_kappa#Significance_and_magnitude), kappa score lower than 20% suggests no agreements between observed accuracy and expected accuracy. This model would be highly encouraged to only explore the relationship between considered features and the label outcome, instead of being used as a prediction model.

## 5.3 Model Intepretation 


In [22]:
feature_imp = pd.DataFrame(rus_clf.best_estimator_.named_steps['rus'].feature_importances_*100,index=X.columns,columns=['Ranking (%)'])
feature_imp['Total count'] = data.sum()
feature_imp.loc[feature_imp['Total count']>12361,'Total count']=12321
feature_imp['Total count']= feature_imp['Total count'].astype(int)
feature_imp = feature_imp.sort_values(by='Ranking (%)',ascending=False)

In [23]:
round(feature_imp[:10],2)

,Ranking (%),Total count
cas_age,26.00,12321
area_speed,8.67,12321
crash_type_Head On,7.67,493
crash_type_Hit Parked Vehicle,4.67,459
unit_movement_Crossing without Control,4.00,276
crash_type_Hit Fixed Object,3.00,1773
lic_type_Unlicenced,2.67,171
cas_gender_Male,2.67,6197
thrown_out_Thrown Out,2.67,110
cas_total,2.33,12321


### 5.3.1 Discussion

The table above shows top 10 features found to be significant in the selected model. With use of visual aids in Data visualisation 03 notebook, we can summarise the effect of these features. 

Causalties with older age and higher total number of casualties taken place at highly speedy area pose higher risk of being fatal in road accidents. Male casualties associated with Head-on collision, hitted fixed object incease the chance of being killed in road accidents.Causalties associated with hitting Parked Vehicle incease the change being survived in road accidents. 

unit_movement_Crossing without Control, lic_type_Unlicenced and thrown_out_Thrown Out are found to have significant effect but their sample sizes are small, suggesting that these variables might not be able to generalise claimed effect on fatality outcome. 

Causalty taken place in country was expected to have higher significant ranking percentage but it was found to have only to have only 1% significant score. This result might be due to impact of highly significant factors such as age and area speed which can capture more variability of the label outcome. 

However, Age has the significant score of 26%, which might be contradicted to what we expected as younger drivers are less experienced than older drivers in reality. The result might also be affected if the drivers have driven for longer distance with longer hours. Unknown factors such as health conditions might explain why older causalties have higher risk of being fatal in accident. External factors such as drinking driving could have substantial impact on the modelling outcome. 

## 5.4 Summary and development for future analysis

RusBoosting classifier was proven to be the best valid model, and age, area speed accident associating with head On collision, and Hitting parked Vehicle were found to be the most significant features to explain the fatality outcome in road accident. However, the low kappa score of selected model does not reflect adequate agreements between observed accuracy and expected accuracy, additional features might need to be added in follow-up analysis to increase more of variability of the data. For example, a measurement of health condition of the causalties can highly influence the outcome of fatality in road accident, as people with heart strokes might be more vulnerable to injury in road accidents, and health history can be added to the model in future analysis. 